# Práctica 12. Evaluación en clasificación

### Grupo 41
Alumnos:
- Óscar Rico Rodríguez
- Jia Hao Yang

In [47]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as mse

### Ejercicio 1: Estimación del valor de las casas
Para la estimación de valor de las casas se compararán dos modelos de regresión como son la regresión lineal vista en la práctica 3 y los árboles de regresión. Estos últimos están implementados en la librería sklearn a través de la clase `DecisionTreeRegressor` [15]. La clase comparte algunos argumentos con la clase `DecisionTreeClassifier` utilizada en la práctica 11. Aunque en este caso no se dispone de la función `plot_tree` ya que es específica para árboles de decisión y no de regresión.
En este ejercicio se realizará:
1. Obtener el rendimiento de árbol de regresión mediante la técnica de división repetida (_holdout repetido_), para los diferentes criterio de división de nodos que posee la clase.
2. Obtener el rendimiento de un modelo de regresión lineal mediante la técnica de división repetida (_holdout repetido_).
3. Comparar los resultados anteriores en función del valor medio y la desviación típica.



In [48]:
# Cargar el dataset
df = pd.read_csv('./precio_casas.csv', sep=';')

##### 1. Obtener el rendimiento de árbol de regresión mediante la técnica de división repetida (holdout repetido), para los diferentes criterio de división de nodos que posee la clase.

In [49]:
criterions_dtr = ['squared_error', 'absolute_error', 'friedman_mse', 'poisson']
data_dtr = []
results_dtr = pd.DataFrame(columns=['squared_error', 'absolute_error', 'friedman_mse', 'poisson'])
for i in range (0, 10):
    X_train, X_test, y_train, y_test = train_test_split(df.loc[:, df.columns != 'Mediana precio'], df['Mediana precio'])
    data_dtr.append([X_train, X_test, y_train, y_test])
    scores = []
    for c in criterions_dtr:
        modelo = DecisionTreeRegressor(criterion= c)
        # Entrenar el modelo
        modelo.fit(X_train, y_train)
        # Predecir
        y_pred = modelo.predict(X_test)
        # Obtener el rendimiento
        scores.append(round(mse(y_test, y_pred) * 100,2))
    results_dtr.loc[i] = scores
results_dtr

,squared_error,absolute_error,friedman_mse,poisson
0,52.14,53.72,53.29,50.75
1,50.50,52.87,50.40,50.88
2,53.39,54.20,53.67,51.39
3,54.91,53.95,53.84,51.57
4,51.81,53.77,52.26,51.92
5,53.64,50.80,53.25,49.16
6,53.57,55.52,53.79,54.58
7,51.87,53.05,50.96,50.26
8,48.92,47.75,49.51,49.92
9,50.80,53.74,50.96,51.80


##### 2. Obtener el rendimiento de un modelo de regresión lineal mediante la técnica de división repetida (holdout repetido).

In [50]:
data_lr = []
results_lr = pd.DataFrame(columns=['Score'])
for i in range (0, 10):
    X_train, X_test, y_train, y_test = train_test_split(df.loc[:, df.columns != 'Mediana precio'], df['Mediana precio'])
    data_lr.append([X_train, X_test, y_train, y_test])
    scores = []
    modelo = LinearRegression()
    # Entrenar el modelo
    modelo.fit(X_train, y_train)
    # Predecir
    y_pred = modelo.predict(X_test)
    # Obtener el rendimiento
    scores.append(round(mse(y_test, y_pred) * 100,2))
    results_lr.loc[i] = scores
results_lr

,Score
0,51.79
1,53.20
2,54.21
3,53.60
4,53.25
5,53.32
6,51.96
7,54.24
8,54.12
9,52.32


##### 3. Comparar los resultados anteriores en función del valor medio y la desviación típica.

In [51]:
Compare = pd.DataFrame(columns=['Model', 'Mean', 'Std'])
for i,d in enumerate(criterions_dtr):
    Compare.loc[i] = [f'DecisionTreeRegressor[{d}]',round(results_dtr.iloc[:, i].mean(),2), round(results_dtr.iloc[:, i].std(),2)]
Compare.loc[i + 1] = ['LinearRegression', round(results_lr.iloc[:, 0].mean(),2), round(results_lr.iloc[:, 0].std(),2)]
Compare

,Model,Mean,Std
0,DecisionTreeRegressor[squared_error],52.15,1.78
1,DecisionTreeRegressor[absolute_error],52.94,2.18
2,DecisionTreeRegressor[friedman_mse],52.19,1.61
3,DecisionTreeRegressor[poisson],51.22,1.47
4,LinearRegression,53.20,0.91


In [59]:
Best_mean_model = Compare.sort_values(by=['Mean']).iloc[0]['Model']
Best_mean = Compare.sort_values(by=['Mean']).iloc[0]['Mean']

Best_std_model = Compare.sort_values(by=['Std']).iloc[0]['Model']
Best_std = Compare.sort_values(by=['Std']).iloc[0]['Std']

print(
f"""
En términos del valor medio, el modelo {Best_mean_model} parece ser el mejor, con un valor medio de {Best_mean}, que es el más bajo entre todos los modelos. Un error cuadrático medio (mean_squared_error) más bajo indica un mejor rendimiento del modelo. 
En cuanto a la desviación típica, el modelo {Best_std_model} es el mejor, con una desviación típica de {Best_std}, lo que indica una menor variabilidad en los resultados.
"""
)


En términos del valor medio, el modelo DecisionTreeRegressor[poisson] parece ser el mejor, con un valor medio de 51.22, que es el más bajo entre todos los modelos. Un error cuadrático medio (mean_squared_error) más bajo indica un mejor rendimiento del modelo. 
En cuanto a la desviación típica, el modelo LinearRegression es el mejor, con una desviación típica de 0.91, lo que indica una menor variabilidad en los resultados.



Tras un cierto número de ejecuciones, hemos podido observar que en la gran mayoría de casos el modelo entrenado mediante la función `DecisionTreeRegressor` usando el criterio _poisson_ es el modelo que mejor media obtiene, siendo así el modelo que muestra un mejor rendimiento. Aunque esto podría ser algo discutible, ya que su desviación estandar en pocos casos es inferior a 2, lo que podría hacer de este un modelo algo inestable. El modelo que mejor desviación estandar ha demostrado tener en este cierto número de ejecuciones ha sido el modelo `LinearRegression`.Para este caso, la decisión debería ser tomada en función de los objetivos del problema.

Por ejemplo, si se considera que es más importante minimizar la media de los errores, se podría escoger el modelo `DecisionTreeRegressor` con el criterio _poisson_ como el mejor, ya que tiene la media promedio más baja. Si, por otro lado, se considera que es más importante tener una menor variabilidad en los errores, se podría escoger el modelo `LinearRegression` como el mejor, ya que tiene la desviación estándar promedio más baja.

`Nota: Esta conclusión ha sido sacada de ejecutar varias veces el código y comprobar los resultados del chunk anterior, puede que en una nueva ejecución estos resultados no se vean reflejados en el chunk anterior, pero sí son los más frecuentes`


El mejor resultado de todos los obtenidos ha sido el siguiente: 

Model:  DecisionTreeRegressor[poisson]

Mean: 50.032

Std: 1.232178

### Ejercicio 2: Implementación de la validación cruzada
Este ejercicio consiste en implementar una función denominada validacion_cv que obtendrá
las particiones siguiendo la técnica de validación cruzada. Para comprobar el funcionamiento de la
función implementada se deberá además, obtener el rendimiento de un árbol de regresión para el
problema de la estimación del precio de las casas.

La declaración de la funcion es la siguiente:

```{python} 
validacion_cv(n, n_particiones=5, mezclar=True, semilla=None)
```

Siendo los argumentos de la función:

- `n`: Es un entero que indica el número de muestras del conjunto de datos en el cual se entrenará
con las particiones resultantes.
- `n_particiones`: Será un entero mayor que 1 que indicará el número de particiones que
compondrán la división por validación cruzada. Por defecto tendrá un valor de 5 particiones.
- `mezclar`: Un valor lógico que indica si se barajan las muestras antes de realizar las particiones. Por defecto tiene el valor True e implica que se barajan. Si se pone a False este argumento significa que las particiones se realizan sin modificar el orden de las muestras.
- `semilla` Un valor entero que se corresponde con la semilla del generador de números
aleatorios utilizado. Por defecto tiene el valor None que indica que no se fija el valor de la
semilla para realizar la mezcla. En caso de que el argumento mezclar sea igual a False,
no se tiene en cuenta el valor de este argumento.

La función debe devolver una lista con tantos elementos como particiones se haya indicado en el
argumento n_particiones. Cada elemento de dicha lista será una tupla compuesta por dos
vectores numpy que contendrán los índices del conjunto de train y test respectivamente.

In [67]:
def validacion_cv(n, n_particiones=5, mezclar=True, semilla=None):
    ret_list = []
    n = [i for i in range(0, n)]
    np.random.seed(semilla)
    if mezclar:
        np.random.shuffle(n)
        particiones = np.array_split(n, n_particiones)
        for arr in particiones:
            ret_list.append([arr, np.setdiff1d(n, arr)])
        return ret_list
    else:
        particiones = np.array_split(n, n_particiones)
        for arr in particiones:
            ret_list.append([arr, np.setdiff1d(n, arr)])
        return ret_list

In [70]:
criterions_dtr2 = ['squared_error', 'absolute_error', 'friedman_mse', 'poisson']
data_dtr2 = []
results_dtr2 = pd.DataFrame(columns=['squared_error', 'absolute_error', 'friedman_mse', 'poisson'])

n = df.shape[0]
for i,d in enumerate( validacion_cv(n = n, mezclar= True)):
    x_train = df.iloc[d[0], 0:8]
    x_test = df.iloc[d[1], 0:8]
    y_train = df.iloc[d[0], 8]
    y_test = df.iloc[d[1], 8]
    data_dtr2.append([x_train, x_test, y_train, y_test])
    scores = []
    for c in criterions_dtr2:
        modelo = DecisionTreeRegressor(criterion= c)
        # Entrenar el modelo
        modelo.fit(x_train, y_train)
        # Predecir
        y_pred = modelo.predict(x_test)
        # Obtener el rendimiento
        scores.append(round(mse(y_test, y_pred) * 100, 2))
    results_dtr2.loc[i] = scores
results_dtr2

,squared_error,absolute_error,friedman_mse,poisson
0,67.30,63.91,67.91,64.90
1,61.45,66.54,63.67,61.44
2,63.28,69.29,63.62,66.13
3,58.52,60.43,59.41,61.59
4,60.73,59.57,61.48,57.31
